## Satellite Data

This script outputs the satellite-based rasters into the Google Earth Engine Cloud.

Inputs:

    - From MAPBIOMAS:
        - secondary forest age
        - land use land cover
        - fire
    - From ESA CCI Biomass:
        - biomass (Mg C/hectare)

Outputs:

    - From MAPBIOMAS:
        - last observed land use type
        - number of years under each land use type
        - number of fires
        - time since last fire
        - fallow period length


In [1]:
import ee
import geemap
from utils import export_image
from utils import map_image

# Authenticate to Earth Engine
try:
  ee.Initialize(project='ee-ana-zonia')
except Exception as e:
  ee.Authenticate()
  ee.Initialize(project='ee-ana-zonia')

first_year = 1985
last_year = 2020
# 1986 - 2019, years included in analysis
years = range((first_year + 1), last_year)

export_age_agbd = False
export_one_hectare_mask = False

mask_1985 = True
history_interval = 5

In [2]:
# import ages from MapBiomas
age = ee.Image('projects/mapbiomas-workspace/public/collection8/mapbiomas_collection80_secondary_vegetation_age_v1')\
    .select('secondary_vegetation_age_2020')

# Load images from MapBiomas Collection 8 for Land Use Land Cover and Burned Area
lulc = ee.Image("projects/mapbiomas-workspace/public/collection8/mapbiomas_collection80_integration_v1") \
    .select([f"classification_{year}" for year in range(first_year, last_year+1)]).byte() \
    .rename([str(year) for year in range(first_year, last_year+1)]).updateMask(age)

fire = ee.Image("projects/mapbiomas-workspace/public/collection7_1/mapbiomas-fire-collection2-annual-burned-coverage-1") \
    .select([f"burned_coverage_{year}" for year in range(first_year, last_year)]).byte() \
    .rename([str(year) for year in range(first_year, last_year)]).updateMask(age)


## Removing pixels with undesired land use categories

Some land use categories are not relevant to the model (such as rocky surfaces or mangroves)

All pixels with **at least one observation of the undesired land use history** are used to make a mask, to leave behind only pixels with occurrences of only desired land use types.


Land use types we are interested in:

    3 = forest
    15 = pasture
    39 = soy
    20 = sugar cane
    40 = rice
    62 = cotton
    41 = other temporary crop
    46 = coffee
    47 = citrus
    35 = palm oil
    48 = other perennial crop
    9 = forest plantationantation

In [3]:
# List the categories that are DESIRED to be maintained
desired_values = ee.List([3, 15, 39, 20, 40, 62, 41, 46, 47, 35, 48, 9, 21])
mask_all_ones = ee.List.repeat(1, desired_values.size())

# For each band, convert pixels with desired land use types to 1 - undesired types to zero
def remap_band(band_name):
    band = lulc.select(ee.String(band_name))
    new_band = band.remap(desired_values, mask_all_ones, 0)
    return new_band.rename(ee.String(band_name))

# Map the function over the band names
remapped_image = lulc.bandNames().map(remap_band)
# make mask by adding all pixels that add up to the total number of years (all pixels with desired categories)
remapped_image = ee.ImageCollection(remapped_image).toBands()
desired_mask = remapped_image.reduce('sum').eq(lulc.bandNames().size().getInfo())

age = age.updateMask(desired_mask)
lulc = lulc.updateMask(desired_mask)
fire = fire.updateMask(desired_mask)

## Biomass - Export age_agbd

Biomass data is in hectares, but mapbiomas data is 30m resolution.

To deal with edge pixels, we reproject biomass values to 10m resolution and then reaggregate to 30m by using the mean (so there is a buffer for land use pixels caught in between two biomass values).

In [4]:
# biomass for 2020 comes from CCI Biomass
biomass = ee.Image("projects/ee-ana-zonia/assets/raw/biomass")
proj = biomass.projection().getInfo()
crs = proj['crs']
crsTransform = proj['transform']

In [5]:
# Reproject to 10m
biomass = biomass.reproject(crs=age.projection(), scale=10)
# Reaggregate to 30m (mean value)
biomass = biomass.reduceResolution(reducer=ee.Reducer.mean()).reproject(crs=age.projection())
# Mask only to regions with age greater than zero (secondary forests)
biomass = biomass.updateMask(age.mask()).int16().rename('agbd')

age_agbd = age.addBands(biomass).addBands(age.pixelLonLat())

if export_age_agbd == True:
    export_image(age_agbd, f"age_agbd")

## Remove isolated pixels - Export one_hectare_mask

In the map, there were isolated pixels, often around the edges of forest patches. These would likely be due to misclassification, or follow different behaviors due to edge effects.

To avoid this issue, a kernel is applied here to include only secondary forest patches that are one hectare or larger.

In [6]:
kernel = ee.Kernel.square(radius = 100, units = 'meters')
# convert non-forest pixels from NA to zero
age_zeroes = age.unmask(0)
# check what is the most frequent value within each hectare - if it's zero, it means the pixel is surrounded by non-forest cover
age_mask = age_zeroes.reduceNeighborhood(reducer = ee.Reducer.mode(), kernel = kernel).gt(1)

if export_one_hectare_mask == True:
    export_image(age_mask, "one_hectare_mask")

## Land Use Land Cover

### Restricting history to n years before abandonment

Since we only have land use history dating back to 1985, the data is censored. In order to restrict history to exclude pixels with possible long histories of anthropogenic use pre-1985 which could not be included in the analysis:

- Make a mask with regrowing pixels that were forested in 1985 (keep only the patches that were forested in 1985, then subsequently deforested, and regrew after that).

- Making a mask including only pixels that have a specified number of bands (hist_interval) between the first and the last occurrence of an anthropogenic pixel (desired extent of land use history to include)


In [7]:
if mask_1985 == True:
    # select pixels that were forested in 1985
    lulc = lulc.updateMask(lulc.select('1985').eq(3))
        
    # for each year, make a mask for all anthropogenic pixels, and multiply by the year to have the
    # pixel value be the year of observation of that land use type.
    non_forest_years = ee.Image()
    for yr in years:
        year = f'{yr}'
        lu_yr = lulc.select(year).neq(3).multiply(yr)
        non_forest_years = non_forest_years.addBands(lu_yr)
    
    # find the minimum and maximum non-zero value per pixel across all bands
    # (first and last year of anthropogenic land use)
    first_anthro = non_forest_years.selfMask().reduce(ee.Reducer.min())
    last_anthro = non_forest_years.reduce(ee.Reducer.max())
    
    # get total of continuous years from the first and last observation of anthropogenic land use
    anthro_gaps = last_anthro.subtract(first_anthro).add(1)

    lulc = lulc.updateMask(anthro_gaps.eq(history_interval).selfMask())


### Total sum of years under each desired land use type

For each desired land use type, the total occurrences per pixel are summed

In [8]:
# Listing land use types that are NOT forest
LU_index = [15, 39, 20, 40, 62, 41, 46, 47, 35, 48, 9, 21]

LU_sum = ee.Image()
for val in LU_index:
  lulc_val = lulc.eq(val)
  num_cells = lulc_val.reduce(ee.Reducer.sum()).rename(f'lulc_sum_{val}')
  LU_sum = LU_sum.addBands(num_cells)

# removes the first (empty) band
LU_sum = LU_sum.slice(1).byte()

### Last Land Use Type

Finds what was the last land use type observed before abandonment.

In [9]:
# for each forest age, mask the lulc of the year immediately preceding abandonment
last_LU = ee.Image()
for yr in years:
    year = f'{yr}'
    lu_yr = lulc.select(year)
    age_mask = age.eq(last_year - yr)
    last_LU = last_LU.addBands(lu_yr.updateMask(age_mask))

# remove first empty band
# merge all images by adding them up
last_LU = last_LU.slice(1).reduce(ee.Reducer.sum()).rename('last_LU')

### Fallow period length

Finds the total number of years that land was left fallow (classified as forest before regrowth)

In [10]:
nat_cover = lulc.eq(3)
total_nat_cover = nat_cover.reduce(ee.Reducer.sum())
fallow = total_nat_cover.subtract(age).rename('fallow')

if mask_1985 == True:
    fallow = fallow.subtract(first_anthro).add(1985)

## Fire

### Total number of fires per pixel

Each burned pixel contains the value of the land use type that burned (such as the desired_values vector specified in the land use section) - unburned pixels are zero.

There are instances of fires:
- after the first observation of anthropogenic land use
- before the first observation of anthropogenic land use (forest fire)
- after the last observation of anthropogenic land use (burn in regrowing forest)

*Note that fire has different transform than lulc, and the projections will be adjusted when exporting.*

### How many years ago was the last fire?

Make one image with 34 bands, each with the number of years from the instance of burn to 2020.
Each pixel is 0 if unburned, and "time since fire" in number of years if it's burned.

In [21]:
# Making a fire mask with pixels of value 1 for burned pixels and 0 otherwise instead:
fire = fire.gt(0)

if mask_1985 == True:
    first_year_anthro = first_year + 1 + history_interval
    years_restricted_history = range((first_year_anthro), last_year)

# Initialize images
num_fires_before_regrowth = ee.Image()
num_fires_after_regrowth = ee.Image()

ts_fire_before_regrowth = ee.Image()
ts_fire_after_regrowth = ee.Image()

num_fires_before_first_anthro = ee.Image()
num_fires_during_anthro = ee.Image()

ts_all_fires = ee.Image()
# for each year, multiply the fire mask by the year to have 
# the pixel value be the year of observation of each fire
for yr in years:
    ts_all_fires = ts_all_fires.addBands(fire.select([f"{yr}"]).multiply(last_year - yr))

def add_fire_bands(image, start_year, end_year, mask, input):
    """
    Adds bands to an image for a range of years, applying a mask and a reducer.
    """
    bands = input.select([f"{year}" for year in range(start_year, end_year)]).updateMask(mask)

    if input == ts_all_fires:
        return image.addBands(bands.reduce(ee.Reducer.lastNonNull()).byte())
    else:
        return image.addBands(bands.reduce(ee.Reducer.sum()).byte())

for yr in years:
    yrs_ago = last_year - yr
    age_mask = age.eq(yrs_ago)
    
    # Add bands for after regrowth
    num_fires_after_regrowth = add_fire_bands(num_fires_after_regrowth, yr, \
                                              last_year, age_mask, fire)
    # Add bands for before regrowth
    num_fires_before_regrowth = add_fire_bands(num_fires_before_regrowth, first_year, \
                                                   yr, age_mask, fire)

    # Add bands for after regrowth
    ts_fire_after_regrowth = add_fire_bands(ts_fire_after_regrowth, yr, \
                                              last_year, age_mask, ts_all_fires)
    # Add bands for before regrowth
    ts_fire_before_regrowth = add_fire_bands(ts_fire_before_regrowth, first_year, \
                                                   yr, age_mask, ts_all_fires)
    
    if mask_1985 == True:
        # Add bands for before first anthro
        num_fires_before_first_anthro = add_fire_bands(num_fires_before_first_anthro, first_year, \
                                                       yr - history_interval, age_mask, fire)
        # Add bands for during anthro
        num_fires_during_anthro = add_fire_bands(num_fires_during_anthro, yr - history_interval, \
                                                 yr, age_mask, fire)



def process_band(band, name):
    """
    Slices the band, reduces it, and renames it.
    """
    return band.slice(1).reduce(ee.Reducer.sum()).rename(name)

# Apply the helper function to each band
num_fires = process_band(num_fires_after_regrowth, 'num_fires_after_regrowth') \
.addBands([process_band(num_fires_before_regrowth, 'num_fires_before_regrowth'), \
            process_band(ts_fire_before_regrowth, 'ts_fire_before_regrowth'), \
            process_band(ts_fire_after_regrowth, 'ts_fire_after_regrowth')])

# Slice and reduce the final image
if mask_1985:
    num_fires = num_fires.addBands([process_band(num_fires_during_anthro, 'num_fires_during_anthro'), \
                                    process_band(num_fires_before_first_anthro, 'num_fires_before_first_anthro')])


vis = {
    'min': 1,
    'max': 30,
    'palette': ['blue', 'red'],
}

map = geemap.Map()
map.addLayer(num_fires.select('ts_fire_before_regrowth'), vis)
map.addLayer(fire)
# map.addLayer(age)
map


EEException: Image.select: Band pattern '1985' did not match any bands. Available bands: [constant, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]

### Mature forest biomass

In [12]:
# Mask the image to keep only pixels with natural vegetation cover in all bands
# 3 = forest
# 6 = flooded forest
ecoregions = (ee.FeatureCollection("RESOLVE/ECOREGIONS/2017").filterBounds(roi)
                .map(lambda feature: feature.intersection(roi)))

mature_cover = lulc.eq(3)
mature_mask = mature_cover.reduce(ee.Reducer.allNonZero())
mature_biomass = biomass.updateMask(mature_mask.mask())


In [13]:
# Compute the mean biomass values for mature forest per ecoregion.
median_mature = mature_biomass.reduceRegions(ecoregions, reducer = ee.Reducer.median(),\
                                             scale = 10000, crs = crs)

# Convert the FeatureCollection to an image.
median_mature = median_mature.reduceToImage(['median'], ee.Reducer.first())



## Export images as asset to Google Earth Engine

In [13]:
frag = ee.Image('projects/ee-ana-zonia/assets/frag_2020')
land_use = LU_sum.addBands([last_LU, fallow, last_fire, num_fires, frag, med_mat])

export_image(land_use, f"land_use_{max_age}_years_one_hectare_1985_forested")

# export_image(biomass, "biomass_masked")
# export_image(median_mature, 'median_mature')
# export_image(mature_biomass, "mature_biomass")


In [17]:


yearly_si = ee.Image("projects/ee-ana-zonia/assets/yearly_si").float()
mean_prec = ee.Image("projects/ee-ana-zonia/assets/mean_prec").float()
climate = yearly_si.addBands([mean_prec])

biomass = ee.Image('projects/ee-ana-zonia/assets/biomass_masked')
total_export = categorical.addBands([land_use, climate, biomass, age]).updateMask(age)
